In [1]:
# !pip install google-cloud-aiplatform[prediction]>=1.16.0 fastapi --user

# Sklearn with Pandas - Custom Prediction Routine to get Merlin Model predictions

Your output should look like this - you are going to use the query model endpoint to create a CPR Endpoing

![](img/merlin-bucket.png)

This is similar to [the other notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb) except we will be using pandas and bigquery

Topics covered
* Training sklearn locally, deploying to endpoint
* Saving data as CSV and doing batch predict from GCS
* Loading data to BQ, using BQ magics
* Running a batch prediction from BQ to BQ

In [2]:
# !gsutil mb -l us-central1 gs://wortz-project-bucket

In [3]:
from datetime import datetime


PROJECT = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
REGION = 'us-central1' 
BUCKET = 'gs://spotify-beam-v3'
REPOSITORY = 'merlin-spotify-cpr'
ARTIFACT_URI = f'{BUCKET}/merlin-processed'
MODEL_DIR = f'{BUCKET}/merlin-processed/query_model_merlin'
PREFIX = 'merlin-spotify'

# New section - preprocessor creation.

In this section we will create a pipeline object that stores a standard scaler 
using the `PipeLine` class is important as it provides a lot of flexibility and conforms to sklearn's framework

## Create a generic sklearn container that returns instances

https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb

**highly recommend reviewing this notebook first as it breaks down the custom predictor interface**

In [4]:
! rm -rf container_code
! mkdir container_code

In [5]:
%%writefile container_code/requirements.txt
fastapi
uvicorn==0.17.6
google-cloud-aiplatform[prediction]>=1.16.0
typing

Writing container_code/requirements.txt


### CPR Template from here https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines

In [11]:
%%writefile container_code/predictor.py

from abc import ABC, abstractmethod
from typing import Any
from google.cloud.aiplatform.prediction.predictor import Predictor
#libs from base image (link to build in notebook 01 top)
import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf


class Predictor(Predictor):
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str) -> None:
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        self._model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query_model_merlin" ))
        self._workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow"))
        self._workflow.remove_inputs(['track_pop_can', 'track_uri_can', 
                            'duration_ms_can', 'track_name_can', 
                            'artist_name_can','album_name_can',
                            'album_uri_can','artist_followers_can',
                            'artist_genres_can','artist_name_can',
                            'artist_pop_can','artist_pop_pl','artist_uri_can',
                            'artists_followers_pl',])            
        
    def preprocess(self, prediction_input: Any) -> Any:
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        dict_input = json.loads(prediction_input)
        #handle different input types, can take a dict or list of dicts
        if type(dict_input) == list:
            pandas_instance = pd.DataFrame.from_dict(dict_input[0], orient='index').T
            if len(dict_input) > 1:
                for ti in dict_input[0:]:
                    pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(ti, orient='index').T)
        if type(dict_input) == dict:
            pandas_instance = pd.DataFrame.from_dict(dict_input, orient='index').T
        else:
            raise Exception("Data must be provided as a dict or list of dicts")

        transformed_inputs = nvt.Dataset(pandas_instance)
        transformed_instance = self._workflow.transform(transformed_inputs)
        batch = mm.sample_batch(transformed_instance, batch_size=128, include_targets=False)
        return batch

    def predict(self, instances: Any) -> Any:
        """Performs prediction.
        Args:
            instances (Any):
                Required. The instance(s) used for performing prediction.
        Returns:
            Prediction results.
        """
        predictions = self._model(instances)
        return predictions

Overwriting container_code/predictor.py


### Build and push container to Artifact Registry
#### Build your container
To build a custom container, we also need to write an entrypoint of the image that starts the model server. However, with the Custom Prediction Routine feature, you don't need to write the entrypoint anymore. Vertex AI SDK will populate the entrypoint with the custom predictor you provide.

In [12]:
# Create the repo if needed for the artifacts

! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [13]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [20]:
# ! pip install google-cloud-aiplatform[prediction]>=1.16.0 --user
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
import os
from container_code.predictor import Predictor

from google.cloud.aiplatform.prediction import LocalModel
base_image = 'us-central1-docker.pkg.dev/hybrid-vertex/workbench/merlin-tensorflow-22.09:latest'

SERVER_IMAGE = "merlin-prediction-cpr"  # @param {type:"string"} 

local_model = LocalModel.build_cpr_model(
    "container_code",
    f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{SERVER_IMAGE}",
    predictor=Predictor,
    # base_image=base_image,
    requirements_path="container_code/requirements.txt"
)

/usr/lib/python3.8/subprocess.py:842: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


DockerError: ('\nDocker failed with error code 1.\nCommand: docker build -t us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr --rm -f- container_code\n', ['docker', 'build', '-t', 'us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr', '--rm', '-f-', 'container_code'], 1)

### Test it out with a locally deployed endpoint
Need to generate credentials to test

In [103]:
local_model.get_serving_container_spec()

image_uri: "us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr"
predict_route: "/predict"
health_route: "/health"

In [104]:
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
                 # 'artist_followers_can': 4339757.0, 
                 # 'artist_genres_can': "'hawaiian hip hop', 'rap'", 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 # 'artist_name_can': 'Russ', 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 # 'artist_pop_can': 82.0, 
                 # 'artist_pop_pl': [82., 80., 90., 87., 65.], 
                 # 'artist_uri_can': 'spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS', 
                 # 'artists_followers_pl': [ 4339757.,  5611842., 15046756., 30713126.,   603837.],  
                 'description_pl': '', 
                 # 'duration_ms_can': 237322.0, 
                 #'duration_ms_songs_pl': [237506., 217200., 219080., 226400., 121739.], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 # 'track_name_can': 'We Just Havent Met Yet', 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 # 'track_pop_can': 57.0, 
                 #'track_pop_pl': [79., 58., 83., 71., 57.],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 # 'track_uri_can': 'spotify:track:0VzDv4wiuZsLsNOmfaUy2W', 
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [105]:
import json
json_instance = json.dumps(TEST_INSTANCE)
json_instance

'{"collaborative": "false", "album_name_pl": ["There\'s Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "artist_genres_pl": ["\'hawaiian hip hop\', \'rap\'", "\'chicago rap\', \'dance pop\', \'pop\', \'pop rap\', \'r&b\', \'southern hip hop\', \'trap\', \'urban contemporary\'", "\'pop\', \'pop r&b\'", "\'dance pop\', \'pop\', \'r&b\'", "\'chill r&b\', \'pop\', \'pop r&b\', \'r&b\', \'urban contemporary\'"], "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\\u00c3\\u00a9", "William Singe"], "description_pl": "", "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}'

### Generate credentials - use your 

Go to the console and search "Service Accounts" from there - select your compute account

Then add a json key and upload back to this notebook, then cange the filename for `CREDENTIALS_FILE` below



In [106]:
CREDENTIALS_FILE = "hybrid-vertex-983c0966dff8.json"

In [107]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=ARTIFACT_URI,
    credential_path=CREDENTIALS_FILE) as local_endpoint:
    health_check_response = local_endpoint.run_health_check()
    prediction = local_endpoint.predict(json_instance)

ERROR:google.cloud.aiplatform.prediction.local_endpoint:Exception during starting serving: ('The health check never succeeded.', '', 1).
ERROR:google.cloud.aiplatform.prediction.local_endpoint:Exception during entering a context: ('The health check never succeeded.', '', 1).


DockerError: ('The health check never succeeded.', '', 1)

#### Only run once to generate creds

## Upload the model to Vertex using new Prediction Route Serving Container

In [ ]:
local_model.push_image() #push to container registry

In [ ]:
from google.cloud import aiplatform

model = local_model.upload(
        display_name='merlin spotify query model',
        artifact_uri=ARTIFACT_URI,
        description='two tower model using merlin models with spotify data',
        labels= {'version': 'v1_00'}, 
              
        sync=True, #false will not bind up your notebook instance with the creation operation
    ) 
# model = aiplatform.Model('projects/679926387543/locations/us-central1/models/5966834099661307904')

In [18]:
endpoint = model.deploy(machine_type="n1-standard-4")
# endpoint = aiplatform.Endpoint('projects/679926387543/locations/us-central1/endpoints/8555880517864521728')

Creating Endpoint
Create Endpoint backing LRO: projects/679926387543/locations/us-central1/endpoints/7051678242322776064/operations/1548668243755925504
Endpoint created. Resource name: projects/679926387543/locations/us-central1/endpoints/7051678242322776064
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/679926387543/locations/us-central1/endpoints/7051678242322776064')
Deploying model to Endpoint : projects/679926387543/locations/us-central1/endpoints/7051678242322776064
Deploy Endpoint model backing LRO: projects/679926387543/locations/us-central1/endpoints/7051678242322776064/operations/5585582359740153856
Endpoint model deployed. Resource name: projects/679926387543/locations/us-central1/endpoints/7051678242322776064


In [19]:
endpoint.predict(instances=[[47.7, 83.1, 38.7], [53.6, 76.1, 24.]])

Prediction(predictions=[[0.79, 0.21], [0.24, 0.76]], deployed_model_id='2882294965424095232', explanations=None)

# You should be able to see the logging ops by searching for `aiplatform.googleapis.com`
+ Make sure you click `show query` slider in case there are other limitations
![](images/log_example.png)

In [20]:
df2 = pd.DataFrame(np.random.randint(0.0,100.0,size=(10,3)), # we will do batch predictions based on this
              index=range(10,20),
              columns=['col1','col2','col3'],
              dtype='float64')

instances_formatted_data = df2.to_numpy().tolist()

predict_response = model.predict(
        request_file=instances_formatted_data,
        headers={"Content-Type": "application/json"},
    )

AttributeError: 'Model' object has no attribute 'predict'

### Expected output
From documentation:
```
array([[0.8 , 0.2 ],
       [0.38, 0.62],
       [0.61, 0.39],
       [0.65, 0.35],
       [0.56, 0.44],
       [0.63, 0.37],
       [0.55, 0.45],
       [0.43, 0.57],
       [0.43, 0.57],
       [0.38, 0.62]])
```

In [ ]:
from google.cloud import storage
import csv

# save the csv with the header, no index
df2.to_csv('df2.csv', index=False)

data_directory = BUCKET + "/data"
storage_path = os.path.join(data_directory, 'df2.csv')
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
blob.upload_from_filename("df2.csv")

In [ ]:
batch_prediction_job = model.batch_predict(
        job_display_name='pandas batch predict job sklearn - VALUES JSON',
        gcs_source=storage_path,
        gcs_destination_prefix=BUCKET+"/predictions",
        machine_type='n1-standard-2',
        instances_format='csv', #This is key to parsing CSV input
        # accelerator_count=accelerator_count,
        # accelerator_type=accelerator_type, #if you want gpus
        starting_replica_count=1,
        max_replica_count=2,
        sync=False,
    )

### When successful you should see this
```
{"instance": [16.0, 64.0, 61.0], "prediction": [0.63, 0.37]}
{"instance": [83.0, 27.0, 87.0], "prediction": [0.35, 0.65]}
{"instance": [96.0, 83.0, 57.0], "prediction": [0.68, 0.32]}
{"instance": [11.0, 62.0, 17.0], "prediction": [0.89, 0.11]}
{"instance": [61.0, 28.0, 1.0], "prediction": [0.36, 0.64]}
```